In [1]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
from Graph import *

In [2]:
graph = Graph(warnings = True)

last_vertex = Vertex(-1, -1, 0, warnings = True)
graph.add_vertices(last_vertex)

for i in range(0,10):
    new_vertex = Vertex(i, i, 0, warnings = True)
    edge = Edge(vertices = [last_vertex, new_vertex], warnings = True)
    graph.add_vertices(new_vertex)
    last_vertex = new_vertex

In [3]:
pp.pprint(graph.vertices)
pp.pprint(graph.edges)
    
for i in range(5):
    cur_vertex = next(iter(graph.vertices))
    cur_vertex.set_graph(None)

    pp.pprint(graph.vertices)
    pp.pprint(graph.edges)
    pp.pri("/n")

{   <Vertex: -1,-1,0>,
    <Vertex: 0,0,0>,
    <Vertex: 1,1,0>,
    <Vertex: 2,2,0>,
    <Vertex: 3,3,0>,
    <Vertex: 4,4,0>,
    <Vertex: 5,5,0>,
    <Vertex: 6,6,0>,
    <Vertex: 7,7,0>,
    <Vertex: 8,8,0>,
    <Vertex: 9,9,0>}
{   <Edge: (<Vertex: 4,4,0>) -> (<Vertex: 5,5,0>)>,
    <Edge: (<Vertex: 0,0,0>) -> (<Vertex: 1,1,0>)>,
    <Edge: (<Vertex: -1,-1,0>) -> (<Vertex: 0,0,0>)>,
    <Edge: (<Vertex: 1,1,0>) -> (<Vertex: 2,2,0>)>,
    <Edge: (<Vertex: 2,2,0>) -> (<Vertex: 3,3,0>)>,
    <Edge: (<Vertex: 3,3,0>) -> (<Vertex: 4,4,0>)>,
    <Edge: (<Vertex: 5,5,0>) -> (<Vertex: 6,6,0>)>,
    <Edge: (<Vertex: 6,6,0>) -> (<Vertex: 7,7,0>)>,
    <Edge: (<Vertex: 7,7,0>) -> (<Vertex: 8,8,0>)>,
    <Edge: (<Vertex: 8,8,0>) -> (<Vertex: 9,9,0>)>}
'/n'
{   <Vertex: 0,0,0>,
    <Vertex: 1,1,0>,
    <Vertex: 2,2,0>,
    <Vertex: 3,3,0>,
    <Vertex: 4,4,0>,
    <Vertex: 5,5,0>,
    <Vertex: 6,6,0>,
    <Vertex: 7,7,0>,
    <Vertex: 8,8,0>,
    <Vertex: 9,9,0>}
{   <Edge: (<Vertex: 4,4,0>) -

In [4]:
print(cur_vertex)
print(cur_vertex.graph)

<Vertex: -1,-1,0>
None
